# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
# Load city.csv into DataFrame
city_df = pd.read_csv("../Output/city.csv", index_col=0)
city_df.head()

,city,lat,lon,max_temp,humidity,cloudiness,wind speed,country,date
0,Castro,-24.79,-50.01,44.58,76,5,3.06,BR,1593692519
1,Havelock,34.88,-76.90,75.99,94,1,3.40,US,1593692724
2,Klaksvík,62.23,-6.59,48.20,70,94,16.11,FO,1593692725
3,Vaini,-21.20,-175.20,71.60,68,75,112.74,TO,1593692478
4,George Town,5.41,100.34,86.00,79,20,4.70,MY,1593692486


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [50]:
# Store latitude and longitude in city_locations
city_locations = city_df[["lat", "lon"]]

# Set humidity as weight
humidity = city_df["humidity"]


In [28]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(city_locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [104]:
# Create new DataFrame to fit weather conditions
hotel_df = city_df.loc[(city_df["max_temp"] < 80) & (city_df["max_temp"] > 70) & (city_df["wind speed"] < 10) 
                                 & (city_df["cloudiness"] == 0)]
hotel_df

,city,lat,lon,max_temp,humidity,cloudiness,wind speed,country,date
172,Chokwé,-24.53,32.98,79.72,33,0,7.07,MZ,1593692943
257,Richards Bay,-28.78,32.04,77.27,35,0,4.09,ZA,1593692995
280,Kalengwa,-13.47,25.00,78.66,23,0,6.89,ZM,1593693021
333,Kununurra,-15.77,128.73,73.40,46,0,4.70,AU,1593693055
338,Sumbe,-11.21,13.84,77.07,66,0,9.73,AO,1593693058
340,Ulaangom,49.98,92.07,76.77,42,0,0.74,MN,1593693060
352,Mufumbwe,-13.68,24.80,78.78,23,0,7.58,ZM,1593693068
464,Namatanai,-3.67,152.43,79.07,77,0,5.84,PG,1593693132
467,Xuchang,34.02,113.82,77.00,83,0,4.47,CN,1593693133
468,Durban,-29.86,31.03,79.00,40,0,6.93,ZA,1593693134


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [105]:
# Add a new coloumn "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

C:\Users\ianli\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,city,lat,lon,max_temp,humidity,cloudiness,wind speed,country,date,Hotel Name
172,Chokwé,-24.53,32.98,79.72,33,0,7.07,MZ,1593692943,
257,Richards Bay,-28.78,32.04,77.27,35,0,4.09,ZA,1593692995,
280,Kalengwa,-13.47,25.00,78.66,23,0,6.89,ZM,1593693021,
333,Kununurra,-15.77,128.73,73.40,46,0,4.70,AU,1593693055,
338,Sumbe,-11.21,13.84,77.07,66,0,9.73,AO,1593693058,
340,Ulaangom,49.98,92.07,76.77,42,0,0.74,MN,1593693060,
352,Mufumbwe,-13.68,24.80,78.78,23,0,7.58,ZM,1593693068,
464,Namatanai,-3.67,152.43,79.07,77,0,5.84,PG,1593693132,
467,Xuchang,34.02,113.82,77.00,83,0,4.47,CN,1593693133,
468,Durban,-29.86,31.03,79.00,40,0,6.93,ZA,1593693134,


In [108]:
# create params dictionary for each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["lat"]
    lng = row["lon"]

    # change location for each iteration 
    params["location"] = f"{lat},{lng}"

    # Get query url for google places api for hotels using nearbysearch
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url keeping original params except location
    hotel_response = requests.get(base_url, params=params)
    
    # convert to json
    hotel_response = hotel_response.json()

    # Exception handling if there are no hotels within parameter search criteria
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_response["results"][0]["name"]
    except (KeyError, IndexError):
        print("No Hotel Found")

hotel_df

C:\Users\ianli\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


No Hotel Found
No Hotel Found


,city,lat,lon,max_temp,humidity,cloudiness,wind speed,country,date,Hotel Name
172,Chokwé,-24.53,32.98,79.72,33,0,7.07,MZ,1593692943,Hotel Sorriso de Caridade
257,Richards Bay,-28.78,32.04,77.27,35,0,4.09,ZA,1593692995,BON Hotel Waterfront Richards Bay
280,Kalengwa,-13.47,25.00,78.66,23,0,6.89,ZM,1593693021,
333,Kununurra,-15.77,128.73,73.40,46,0,4.70,AU,1593693055,Freshwater East Kimberley Apartments | Kununur...
338,Sumbe,-11.21,13.84,77.07,66,0,9.73,AO,1593693058,Hotel Kalunda
340,Ulaangom,49.98,92.07,76.77,42,0,0.74,MN,1593693060,Grand restaurant & hotel
352,Mufumbwe,-13.68,24.80,78.78,23,0,7.58,ZM,1593693068,
464,Namatanai,-3.67,152.43,79.07,77,0,5.84,PG,1593693132,Namatanai Hotel
467,Xuchang,34.02,113.82,77.00,83,0,4.47,CN,1593693133,Rebecca Hotel
468,Durban,-29.86,31.03,79.00,40,0,6.93,ZA,1593693134,Southern Sun Elangeni & Maharani


In [109]:
# Drop cities with no hotels
hotel_df = hotel_df.dropna()
hotel_df = hotel_df.rename(columns={"city":"City", "country":"Country"})
hotel_df

,City,lat,lon,max_temp,humidity,cloudiness,wind speed,Country,date,Hotel Name
172,Chokwé,-24.53,32.98,79.72,33,0,7.07,MZ,1593692943,Hotel Sorriso de Caridade
257,Richards Bay,-28.78,32.04,77.27,35,0,4.09,ZA,1593692995,BON Hotel Waterfront Richards Bay
280,Kalengwa,-13.47,25.00,78.66,23,0,6.89,ZM,1593693021,
333,Kununurra,-15.77,128.73,73.40,46,0,4.70,AU,1593693055,Freshwater East Kimberley Apartments | Kununur...
338,Sumbe,-11.21,13.84,77.07,66,0,9.73,AO,1593693058,Hotel Kalunda
340,Ulaangom,49.98,92.07,76.77,42,0,0.74,MN,1593693060,Grand restaurant & hotel
352,Mufumbwe,-13.68,24.80,78.78,23,0,7.58,ZM,1593693068,
464,Namatanai,-3.67,152.43,79.07,77,0,5.84,PG,1593693132,Namatanai Hotel
467,Xuchang,34.02,113.82,77.00,83,0,4.47,CN,1593693133,Rebecca Hotel
468,Durban,-29.86,31.03,79.00,40,0,6.93,ZA,1593693134,Southern Sun Elangeni & Maharani


In [110]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [111]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

# create marker layer
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)

# Add marker layer to heat layer
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))